In [177]:
from models.Trainer import *
dataset='mnist'
class_num=10

net_glob,train_loder,test_loader=get_net_and_loader(dataset=dataset)

traindata=train_loder.dataset.train_data
targetdata=train_loder.dataset.targets

import os
from PIL import Image
from tqdm import tqdm

def show(x,i):
    #plt.figure(dpi=80)
    #data=np.transpose(x.cpu().numpy(), (1,2,0))
    data=x.cpu().numpy()
    # plt.imshow(data,cmap='gray') 
    # plt.axis('off')
    # plt.show()
    im = Image.fromarray(data)
    im.save(i)
    
dir_name='data/{}/train'.format(dataset)
    
for i in range(class_num):
    child_dir_name='{}/{}/'.format(dir_name,i)
    if not os.path.isdir(child_dir_name):
        mkdir_p(child_dir_name)
    
for i in tqdm(range(len(traindata))):
    #show(traindata[i],'{}/{}/{}_{}.jpg'.format(dir_name,targetdata[i],targetdata[i],i))
    pass


/data/chengbin/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
100%|██████████| 60000/60000 [00:00<00:00, 2456496.50it/s]


In [178]:
from models.sampling import *

class DatasetSplit(Dataset):
    def __init__(self, traindata,targetdata, idxs):
        self.traindata = traindata
        self.targetdata= targetdata
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        print(self.idxs[item])
        image, label = self.traindata[self.idxs[item]],self.targetdata[self.idxs[item]]
        return image, label
    
def save(dataset,dir_name):
    traindata = dataset.traindata
    targetdata= dataset.targetdata
    if not os.path.isdir(dir_name):
        mkdir_p(dir_name)
    
    for i in range(class_num):
        child_dir_name='{}/{}/'.format(dir_name,i)
        if not os.path.isdir(child_dir_name):
            mkdir_p(child_dir_name)
    
    for i in tqdm(range(len(traindata))):
        data=traindata[i].cpu().numpy()
        im = Image.fromarray(data)
        im.save(i)
        show(traindata[i],'{}/{}/{}_{}.jpg'.format(dir_name,targetdata[i],targetdata[i],i))

dict_users = mnist_iid(traindata, 4)

for idx in dict_users:
    x=DatasetSplit(traindata,targetdata, dict_users[idx])
    dir_name='test_data/{}/train/part_{}'.format(dataset,idx)
    save(x,dir_name)

NameError: name 'os' is not defined

In [110]:
import socket
sock=socket.socket()
sock.connect(("localhost",22))

In [179]:
import collections
import pandas as pd
from io import StringIO
import pickle
w=net_glob.cpu().state_dict()
w=dict(w)

#data = dict({key:value.numpy() for key,value in w.items()})
print(w)
w1=pickle.dumps(w)
print(w1)
w2=pickle.loads(w1) 
print(w2)

net_glob.load_state_dict(w2)


{'layer_input.weight': tensor([[ 0.0176,  0.0035,  0.0196,  ...,  0.0172, -0.0307,  0.0200],
        [ 0.0169,  0.0042,  0.0319,  ..., -0.0014,  0.0216,  0.0319],
        [ 0.0329, -0.0229,  0.0342,  ..., -0.0240,  0.0119,  0.0335],
        ...,
        [-0.0097,  0.0177,  0.0049,  ...,  0.0066, -0.0258, -0.0285],
        [-0.0184, -0.0206, -0.0048,  ...,  0.0179,  0.0046, -0.0033],
        [ 0.0170,  0.0185,  0.0227,  ...,  0.0091,  0.0085,  0.0322]]), 'layer_input.bias': tensor([ 2.8417e-02, -2.3120e-02, -5.4224e-03, -2.6098e-03,  3.3031e-02,
        -2.2331e-02, -9.3515e-03, -2.0012e-02, -4.7981e-03,  1.4961e-02,
        -1.7393e-02, -2.2846e-02, -1.0369e-02,  1.2111e-03,  3.6655e-03,
        -5.8020e-03,  6.3449e-05, -3.1955e-03, -2.9781e-02, -2.8135e-02,
         6.4125e-03,  1.5282e-02, -3.0787e-02,  2.3981e-02,  2.0747e-02,
         2.1618e-02, -1.3203e-03, -2.4715e-02,  2.9159e-02, -1.8094e-02,
        -2.9479e-02,  1.5935e-02, -2.0851e-02, -1.9833e-02,  1.5932e-02,
         2.

<All keys matched successfully>

In [169]:
import pickle

In [176]:
test_loader.dataset.

/data/chengbin/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,